In [1]:
import os
import pandas as pd
from Bio import SeqIO, Entrez
from Bio.Blast import NCBIWWW, NCBIXML
import requests


In [2]:
Entrez.email = "u7457260@anu.edu.au"

def read_fasta_file(file_path):
    sequences = list(SeqIO.parse(file_path, "fasta"))
    return sequences

def blast_sequence(sequence, program="blastx", database="nr", e_value=0.001, max_length=10000):
    result_handles = []
    for i in range(0, len(sequence), max_length):
        subseq = sequence[i:i+max_length]
        result_handle = NCBIWWW.qblast(program, database, subseq, hitlist_size=10, expect=e_value)
        result_handles.append(result_handle)
    return result_handles

def parse_blast_result(blast_result, e_value_threshold=1e-3, max_results=10):
    blast_records = NCBIXML.read(blast_result)
    best_hits = []

    for alignment in blast_records.alignments[:max_results]:
        for hsp in alignment.hsps:
            if hsp.expect <= e_value_threshold:
                best_hits.append((alignment.accession, hsp.expect))
    return best_hits

def get_kegg_pathways(gene_id):
    url = f"https://www.kegg.jp/dbget-bin/www_bget?{gene_id}"
    response = requests.get(url)

    pathways = []
    if "Pathway" in response.text:
        for line in response.text.split("\n"):
            if "<td><a href=\"/dbget-bin/www_bget?" in line:
                pathway_id = line.split(">")[-1].split("<")[0]
                pathway_name = line.split("</a>")[-1].strip()
                pathways.append((pathway_id, pathway_name))
    return pathways

def get_protein_name_by_gene_id(gene_id):
    search_result = Entrez.esearch(db="protein", term=f"{gene_id}[Accession]")
    result_ids = Entrez.read(search_result)["IdList"]

    protein_names = []
    for result_id in result_ids:
        protein_summary = Entrez.esummary(db="protein", id=result_id)
        protein_info = Entrez.read(protein_summary)[0]
        protein_names.append(protein_info["Title"])

    return protein_names

def main(input_folder, file_name):
    summary_data = []

    file_path = os.path.join(input_folder, file_name)
    sequences = read_fasta_file(file_path)

    for sequence in sequences:
        print(f"Processing {sequence.id} in {file_name}")
        blast_results = blast_sequence(sequence.seq)
        for blast_result in blast_results:
            best_hits = parse_blast_result(blast_result)

            for gene_id, e_value in best_hits:
                pathways = get_kegg_pathways(gene_id)
                protein_names = get_protein_name_by_gene_id(gene_id)

                for protein_name in protein_names:
                    for pathway_id, pathway_name in pathways:
                        summary_data.append({
                            "Bacteria": file_name,
                            "GeneID": gene_id,
                            "Protein": protein_name,
                            "PathwayID": pathway_id,
                            "PathwayName": pathway_name
                        })

    summary_df = pd.DataFrame(summary_data)
    output_file = f"genome/{file_name}_summary.csv"
    summary_df.to_csv(output_file, index=False)


input_folder = "genome"
filenames_file = "genome/name_list.TXT"

with open(filenames_file, 'r', encoding='ISO-8859-1') as file:
    filenames = [line.strip() for line in file.readlines()]

for file_name in filenames:
    main(input_folder, file_name)    

Processing CP000975.1 in GCA_000019665.fna
